# Step 4: Generate Answer from Context

This notebook tests the main generation node. It takes the user's question and the list of filtered, relevant documents and passes them to the LLM to synthesize a final answer.

In [1]:
import os
from langchain_community.llms import Ollama
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
# --- Configuration ---
MODEL_NAME = "llama3"
llm = Ollama(model=MODEL_NAME, temperature=0)

/var/folders/90/p8kgtvm572d51cr90k19v_580000gn/T/ipykernel_64170/916712105.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=MODEL_NAME, temperature=0)


In [3]:
def generate_answer(state: dict, llm) -> dict:
    """
    Generates an answer using the LLM with context.
    This function is defined locally for experimentation.
    """
    print("---GENERATING ANSWER WITH CONTEXT---")
    question = state['question']
    documents = state['documents']
    
    prompt_template = """
    You are an assistant for question-answering tasks for oil and gas emissions compliance.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Be concise and provide the answer based only on the provided context.

    Question: {question}
    Context: {context}

    Answer:
    """
    prompt = PromptTemplate(template=prompt_template, input_variables=["question", "context"])
    
    # Takes a dictionary with a "question" and "context".
    # formats it into a prompt.
    # Sends the prompt to the language model to get a response.
    # Parses the model's response to get a clean string as the final output.
    
    rag_chain = prompt | llm | StrOutputParser()
    
    context_str = "\n\n".join([d.page_content for d in documents])
    generation = rag_chain.invoke({"question": question, "context": context_str})
    return {"generation": generation}

In [4]:
# --- Mock State ---
# We will simulate the state after the 'grade_documents' node has run.
question = "What are the requirements for monitoring a flare's pilot flame?"

context_doc = Document(
    page_content="The presence of a flare pilot flame must be continuously monitored using a thermocouple or any other equivalent device. If the pilot flame is extinguished, an alarm must be triggered, and corrective action must be taken within 5 minutes.",
    metadata={"source": "sample_epa_regulation.txt"}
)

state = {
    "question": question,
    "documents": [context_doc]
}


In [5]:
result = generate_answer(state, llm)

print("--- Generated Answer ---")
print(result['generation'])

---GENERATING ANSWER WITH CONTEXT---
--- Generated Answer ---
According to the provided context, the requirements for monitoring a flare's pilot flame are:

* Continuous monitoring using a thermocouple or any other equivalent device.
* An alarm must be triggered if the pilot flame is extinguished.
* Corrective action must be taken within 5 minutes.
